In [ ]:
def get_weights(model_xml, as_dict=False):
    from openvino.inference_engine import IECore
    ie = IECore()
    model_bin = model_xml.replace('xml', 'bin')
    net = ie.read_network(model=model_xml, weights=model_bin)
    weights = []
    ldict   = {}
    for k in net.layers.keys():
        #print(k, net.layers[k].shape)
        if 'weights' in net.layers[k].weights:
            l = net.layers[k]
            w = l.weights['weights']
            #print(k, l.shape, w.shape)
            weights.append(w)
            ldict[k] = l
    if as_dict: return ldict
    return weights

def calc_sparsity(w):
    import numpy as np
    return np.sum(w == 0)/w.flatten().shape[0]

def weight_hist(model_xml):
    import numpy as np
    import matplotlib.pyplot as plt
    weights = get_weights(model_xml)
    w = np.concatenate(weights)

    hist, bins, _ = plt.hist(w, bins=50)
    plt.cla()
    logbins = np.linspace(np.mean(w) - 3*np.std(w), np.mean(w) + 3*np.std(w),len(bins))

    plt.hist(w, bins=logbins)
    sparsity = 100*calc_sparsity(w)
    plt.yscale('log', nonposy='clip')
    plt.title('%s histogram sparsity=%0.3f%%' % (model_xml, sparsity))
    plt.show()
    
def sparse_summary(model_xml, threshold=0):
    for k, l in get_weights(model_xml, as_dict=True).items():
        w = l.weights['weights']
        s = calc_sparsity(w)
        if s > threshold:
            print(k, l.shape, w.shape, 'Sparsity: %0.2f' % calc_sparsity(w))

In [ ]:
# Show overall weights
#model_xml = 'model32.xml'
model_xml = 'intel/resnet-50-int8-sparse-v1-tf-0001.xml'
weight_hist(model_xml)

In [ ]:
sparse_summary(model_xml, 0.3)

In [ ]:
def sparse_weight_viz(model_xml, threshold=0):
    import matplotlib.pyplot as plt
    import numpy as np
    for k, l in get_weights(model_xml, as_dict=True).items():
        w = l.weights['weights']
        s = calc_sparsity(w)
        if s > threshold:
            print(k, l.shape, w.shape, 'Sparsity: %0.2f' % calc_sparsity(w))
            sz = int(np.ceil(np.sqrt(w.shape)))
            img = np.zeros((sz*sz))
            img[:w.shape[0]] = w
            img = np.reshape(img, (sz, sz))
            mask = img == 0
            img[mask] = 1
            img[~mask] = 0
            plt.imshow(img, cmap='gray')
            plt.show()
            print(img.shape, img.min(), img.mean(), img.max())

In [ ]:
sparse_weight_viz(model_xml, 0.3)